Cargamos las imágenes en un array para tenerlas disponibles luego

In [ ]:
# Nos descargamos el dataset de OpenDataSoft
!wget -O "airbnb-listings.cs

Y a partir de aqui es vuestro trabajo!!!

Pasos Explicados de lo que voy a hacer: (omito la carga y limpieza previa)
    Carga de datos y exploración de los mismos:
    Primero, debemos cargar los datos del dataset de Airbnb y explorarlos para entender la estructura de los datos, identificar valores perdidos, etc.

    Preprocesamiento de los datos:
    A continuación, realizamos el preprocesamiento de los datos, incluyendo la normalización de los datos numéricos y categóricos, la selección de las variables relevantes, la asignación de valores perdidos, y la separación de los datos en conjuntos de entrenamiento y prueba.

    Modelo basado en imágenes:
    Para construir el modelo basado en imágenes, primero necesitamos normalizar los valores de las imágenes, separar los datos en conjuntos de entrenamiento y prueba, y construir el modelo convolucional utilizando una de las opciones mencionadas anteriormente (fine-tuning, transfer learning o from scratch). Luego, compilamos y entrenamos el modelo optimizando los hiperparámetros y evitando el overfitting con técnicas como Dropout, max-pooling, regularización, etc.

    Modelo basado en 1D:
    Para construir el modelo basado en 1D, primero necesitamos seleccionar las variables numéricas relevantes, normalizar los datos, y construir un modelo de red neuronal 1D. Luego, compilamos y entrenamos el modelo optimizando los hiperparámetros y evitando el overfitting con técnicas como Dropout, regularización, etc.

    Modelo híbrido:
    Finalmente, podemos construir un modelo híbrido combinando los dos modelos previamente construidos. Podemos realizar el modelo híbrido utilizando alguna de las técnicas mencionadas anteriormente, como early fusion o late fusion.

In [ ]:
import pandas as pd

# Cargar el dataset
#!wget -O "airbnb-listings.csv" "https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B"
airbnb_data = pd.read_csv('airbnb.csv')

# Explorar los datos
print(airbnb_data.head())
print(airbnb_data.describe())

#Con esto vemos los datos que tenemos columnas y demas

In [ ]:
#Normalización del valor a predecir:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
airbnb_data['precio_normalizado'] = scaler.fit_transform(airbnb_data[['precio']])

#Selección de las variables que son de interés:

features = ['imagen', 'barrio', 'num_habitaciones', 'num_baños', 'wifi', 'tv', 'aire_acondicionado', 'precio_normalizado']
airbnb_data = airbnb_data[features]

# Procesamiento de las variables numéricas y categóricas (normalización de datos):
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Normalización de las variables numéricas
num_features = ['num_habitaciones', 'num_baños']
num_scaler = StandardScaler()
airbnb_data[num_features] = num_scaler.fit_transform(airbnb_data[num_features])

# Codificación de las variables categóricas
cat_features = ['barrio']
cat_encoder = LabelEncoder()
airbnb_data[cat_features] = cat_encoder.fit_transform(airbnb_data[cat_features])

#Búsqueda y asignación de valores perdidos:

airbnb_data.isna().sum()
airbnb_data = airbnb_data.fillna(0)

# Split de los datos

from sklearn.model_selection import train_test_split

X = airbnb_data.drop(['precio_normalizado'], axis=1)
y = airbnb_data['precio_normalizado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Generación del modelo (optimización del número de densas, el número de neuronas):

# Generación del modelo basado en imágenes
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_shape = (200, 200, 3)

model_img = Sequential()
model_img.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model_img.add(MaxPooling2D(pool_size=(2, 2)))
model_img.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_img.add(MaxPooling2D(pool_size=(2, 2)))
model_img.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_img.add(MaxPooling2D(pool_size=(2, 2)))
model_img.add(Flatten())
model_img.add(Dense(256, activation='relu'))
model_img.add(Dropout(0.5))
model_img.add(Dense(1, activation='linear'))

# Generación del modelo basado en datos 1D
model_1d = Sequential()
model_1d.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model_1d.add



Una vez que hemos procesado las variables numéricas y categóricas, es momento de generar el modelo y optimizar el número de capas densas y neuronas.

Para construir el modelo, crearemos un modelo basado en 1D, donde las entradas serán los datos preprocesados y la salida será la variable objetivo (en este caso, el precio de la habitación).

A continuación, intentare acabar dos capas densas y 64 neuronas cada una.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

model.summary()


La primera capa densa tiene 64 neuronas, función de activación ReLU y una entrada de tamaño X_train.shape[1]. 

La segunda capa densa también tiene 64 neuronas y función de activación ReLU. 

La última capa densa tiene una sola neurona, y la función de activación es lineal.



In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)


Hemos utilizado la función de pérdida mean_squared_error, que es adecuada para problemas de regresión, y el optimizador Adam con un learning rate de 0.001. También definimos el número de épocas y el tamaño de lote (batch size) para el entrenamiento.

Finalmente, el modelo puede ser evaluado en el conjunto de datos de prueba y se puede calcular la precisión de la predicción. También se puede ajustar el modelo para mejorar su precisión y hacer predicciones más precisas.

(Pendiente de acabar he dejado todo a medias)


In [ ]:
## Para las imagenes

#Entrenamiento del modelo de imágenes:
image_model.fit(train_image_data, train_labels, epochs=50, batch_size=32, validation_data=(val_image_data, val_labels))

#Entrenamiento del modelo 1D:
numeric_model.fit(train_numeric_data, train_labels, epochs=50, batch_size=32, validation_data=(val_numeric_data, val_labels))

# Fusión de modelos:

combined_input = concatenate([image_model.output, numeric_model.output])
x = Dense(64, activation='relu')(combined_input)
x = Dense(32, activation='relu')(x)
x = Dense(1, activation='linear')(x)
combined_model = Model(inputs=[image_model.input, numeric_model.input], outputs=x)
combined_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
combined_model.fit([train_image_data, train_numeric_data], train_labels, epochs=50, batch_size=32, validation_data=([val_image_data, val_numeric_data], val_labels))
